In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

## Detection functions

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)  
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

## For creating files

In [37]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('D:\\1. BU\\CS677\\Projects\\Bolatbek_Amiyev_Project\\trainingData') 

# Actions that we try to detect
actions = np.array(['Hello', 'My name is Bolatbek', 'Thanks'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [38]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## For capturing train data

In [39]:
cap = cv2.VideoCapture(0)
# Mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Loop through actions
    for action in actions:
        # Loop through sequences videos
        for sequence in range(no_sequences):
            # Loop through video length sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)


                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV', image)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break
                if cv2.waitKey(10) & 0xFF == ord('z'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

## For labeling actions

In [40]:
label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

## Spliting train test parts

In [41]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## Create logs for model

In [42]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

## Create model

In [43]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [44]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 3s 49ms/step - loss: 1.6521 - categorical_accuracy: 0.2941
Epoch 2/2000
3/3 [==============================] - 0s 42ms/step - loss: 1.6216 - categorical_accuracy: 0.2235
Epoch 3/2000
3/3 [==============================] - 0s 42ms/step - loss: 3.2140 - categorical_accuracy: 0.3059
Epoch 4/2000
3/3 [==============================] - 0s 41ms/step - loss: 1.6055 - categorical_accuracy: 0.3059
Epoch 5/2000
3/3 [==============================] - 0s 40ms/step - loss: 2.2481 - categorical_accuracy: 0.2118
Epoch 6/2000
3/3 [==============================] - 0s 42ms/step - loss: 5.0334 - categorical_accuracy: 0.3059
Epoch 7/2000
3/3 [==============================] - 0s 42ms/step - loss: 6.0566 - categorical_accuracy: 0.3529
Epoch 8/2000
3/3 [==============================] - 0s 41ms/step - loss: 18.7791 - categorical_accuracy: 0.3765
Epoch 9/2000
3/3 [==============================] - 0s 42ms/step - loss: 5.2323 - categorical_accuracy: 0.3294


3/3 [==============================] - 0s 40ms/step - loss: 0.4634 - categorical_accuracy: 0.8353
Epoch 147/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.4004 - categorical_accuracy: 0.8471
Epoch 148/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.3984 - categorical_accuracy: 0.8235
Epoch 149/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.3616 - categorical_accuracy: 0.8941
Epoch 150/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.4083 - categorical_accuracy: 0.8353
Epoch 151/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.8110 - categorical_accuracy: 0.6471
Epoch 152/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.6423 - categorical_accuracy: 0.7529
Epoch 153/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.4308 - categorical_accuracy: 0.8353
Epoch 154/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.4386 - categorical_accuracy: 0.835

3/3 [==============================] - 0s 43ms/step - loss: 0.1832 - categorical_accuracy: 0.9176
Epoch 291/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.1759 - categorical_accuracy: 0.9176
Epoch 292/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.1892 - categorical_accuracy: 0.9412
Epoch 293/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.1587 - categorical_accuracy: 0.9412
Epoch 294/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.2051 - categorical_accuracy: 0.9176
Epoch 295/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.2339 - categorical_accuracy: 0.9059
Epoch 296/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.2162 - categorical_accuracy: 0.9529
Epoch 297/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.2521 - categorical_accuracy: 0.9176
Epoch 298/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.1828 - categorical_accuracy: 0.941

3/3 [==============================] - 0s 47ms/step - loss: 0.5586 - categorical_accuracy: 0.8235
Epoch 435/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.4553 - categorical_accuracy: 0.8824
Epoch 436/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.5711 - categorical_accuracy: 0.8118
Epoch 437/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.5802 - categorical_accuracy: 0.7765
Epoch 438/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.8717 - categorical_accuracy: 0.6824
Epoch 439/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.5679 - categorical_accuracy: 0.8000
Epoch 440/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.6523 - categorical_accuracy: 0.6588
Epoch 441/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.6395 - categorical_accuracy: 0.8000
Epoch 442/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.5572 - categorical_accuracy: 0.847

3/3 [==============================] - 0s 46ms/step - loss: 0.5467 - categorical_accuracy: 0.7294
Epoch 579/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.6003 - categorical_accuracy: 0.6824
Epoch 580/2000
3/3 [==============================] - 0s 47ms/step - loss: 0.5638 - categorical_accuracy: 0.6941
Epoch 581/2000
3/3 [==============================] - 0s 52ms/step - loss: 0.5490 - categorical_accuracy: 0.6941
Epoch 582/2000
3/3 [==============================] - 0s 48ms/step - loss: 0.5339 - categorical_accuracy: 0.7647
Epoch 583/2000
3/3 [==============================] - 0s 54ms/step - loss: 0.5061 - categorical_accuracy: 0.6824
Epoch 584/2000
3/3 [==============================] - 0s 49ms/step - loss: 0.5003 - categorical_accuracy: 0.7059
Epoch 585/2000
3/3 [==============================] - 0s 51ms/step - loss: 0.5067 - categorical_accuracy: 0.7294
Epoch 586/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.4959 - categorical_accuracy: 0.823

...

3/3 [==============================] - 0s 43ms/step - loss: 1.0029 - categorical_accuracy: 0.4941
Epoch 1659/2000
3/3 [==============================] - 0s 48ms/step - loss: 0.9617 - categorical_accuracy: 0.4471
Epoch 1660/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.9599 - categorical_accuracy: 0.4706
Epoch 1661/2000
3/3 [==============================] - 0s 47ms/step - loss: 0.9432 - categorical_accuracy: 0.4941
Epoch 1662/2000
3/3 [==============================] - 0s 48ms/step - loss: 0.9203 - categorical_accuracy: 0.4941
Epoch 1663/2000
3/3 [==============================] - 0s 49ms/step - loss: 0.9145 - categorical_accuracy: 0.5059
Epoch 1664/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.8963 - categorical_accuracy: 0.5294
Epoch 1665/2000
3/3 [==============================] - 0s 48ms/step - loss: 0.9171 - categorical_accuracy: 0.4824
Epoch 1666/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.9219 - categorical_accurac

3/3 [==============================] - 0s 44ms/step - loss: 1.1086 - categorical_accuracy: 0.4000
Epoch 1731/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.9345 - categorical_accuracy: 0.5176
Epoch 1732/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.9359 - categorical_accuracy: 0.5176
Epoch 1733/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.8897 - categorical_accuracy: 0.5294
Epoch 1734/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.8705 - categorical_accuracy: 0.5059
Epoch 1735/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.8318 - categorical_accuracy: 0.6706
Epoch 1736/2000
3/3 [==============================] - 0s 48ms/step - loss: 0.8096 - categorical_accuracy: 0.6588
Epoch 1737/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.7992 - categorical_accuracy: 0.6471
Epoch 1738/2000
3/3 [==============================] - 0s 46ms/step - loss: 0.7901 - categorical_accurac

3/3 [==============================] - 0s 45ms/step - loss: 0.5805 - categorical_accuracy: 0.7529
Epoch 1803/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.6648 - categorical_accuracy: 0.7294
Epoch 1804/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.7606 - categorical_accuracy: 0.5647
Epoch 1805/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.7206 - categorical_accuracy: 0.6235
Epoch 1806/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.7210 - categorical_accuracy: 0.5765
Epoch 1807/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.7644 - categorical_accuracy: 0.6235
Epoch 1808/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.7433 - categorical_accuracy: 0.5529
Epoch 1809/2000
3/3 [==============================] - 0s 44ms/step - loss: 0.6594 - categorical_accuracy: 0.6471
Epoch 1810/2000
3/3 [==============================] - 0s 51ms/step - loss: 0.6522 - categorical_accurac

3/3 [==============================] - 0s 42ms/step - loss: 0.3801 - categorical_accuracy: 0.8706
Epoch 1875/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.3746 - categorical_accuracy: 0.8824
Epoch 1876/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.4598 - categorical_accuracy: 0.7765
Epoch 1877/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.4617 - categorical_accuracy: 0.8588
Epoch 1878/2000
3/3 [==============================] - 0s 42ms/step - loss: 0.4546 - categorical_accuracy: 0.6706
Epoch 1879/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.4440 - categorical_accuracy: 0.8706
Epoch 1880/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.4463 - categorical_accuracy: 0.7294
Epoch 1881/2000
3/3 [==============================] - 0s 51ms/step - loss: 0.4546 - categorical_accuracy: 0.8706
Epoch 1882/2000
3/3 [==============================] - 0s 43ms/step - loss: 0.5867 - categorical_accurac

3/3 [==============================] - 0s 50ms/step - loss: 0.4241 - categorical_accuracy: 0.8706
Epoch 1947/2000
3/3 [==============================] - 0s 72ms/step - loss: 0.3526 - categorical_accuracy: 0.8235
Epoch 1948/2000
3/3 [==============================] - 0s 47ms/step - loss: 0.2910 - categorical_accuracy: 0.8235
Epoch 1949/2000
3/3 [==============================] - 0s 57ms/step - loss: 0.3375 - categorical_accuracy: 0.8824
Epoch 1950/2000
3/3 [==============================] - 0s 59ms/step - loss: 0.2615 - categorical_accuracy: 0.8824
Epoch 1951/2000
3/3 [==============================] - 0s 56ms/step - loss: 0.2390 - categorical_accuracy: 0.9059
Epoch 1952/2000
3/3 [==============================] - 0s 67ms/step - loss: 0.2102 - categorical_accuracy: 0.9059
Epoch 1953/2000
3/3 [==============================] - 0s 64ms/step - loss: 0.2303 - categorical_accuracy: 0.9059
Epoch 1954/2000
3/3 [==============================] - 0s 66ms/step - loss: 0.1949 - categorical_accurac

## Statistics results

In [45]:
res = model.predict(X_test)

1/1 [==============================] - 0s 344ms/step


In [50]:
y_pred = model.predict(X_test)
y_true = np.argmax(y_test, axis=1).tolist()
y_pred = np.argmax(y_pred, axis=1).tolist()

1/1 [==============================] - 0s 28ms/step


In [51]:
multilabel_confusion_matrix(y_true, y_pred)

array([[[0, 0],
        [0, 5]]], dtype=int64)

In [52]:
accuracy_score(y_true, y_pred)

1.0

## Saving model

In [49]:
model.save('action.h5')